In [9]:
import glob
import pandas as pd
from tensorflow import keras
import numpy as np
import os 
import matplotlib.pylab as plt
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.layers import TimeDistributed, Conv2D, Conv2DTranspose, MaxPooling2D, AveragePooling2D, BatchNormalization, concatenate, Input, ConvLSTM2D, Reshape, Conv3D, Flatten, LSTM, GRU, Dense,Dropout, Add
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.models import Sequential, load_model
from sklearn.utils import shuffle

In [18]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size=32, dim=(120,120), n_channels=1, n_timesteps = 4, shuffle=True, augment_data = True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_timesteps = n_timesteps 
        self.shuffle = shuffle
        self.augment_data = augment_data 
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' 
        
        if self.augment_data == True:  # only augment data when training 
            # Initialization
            X = np.empty((self.batch_size*5, 120, 120, 4))
            y = np.empty((self.batch_size*5, 120, 120, 1)) 

            # Generate data
            for i, ID in enumerate(list_IDs_temp):
                data = np.load('./storage/precipitation/train/' + ID)
                # Store sample
                x_data = data[:,:,:4] 
                y_data = data[:,:,-1].reshape((120,120,1)) 
                
                X[i,] = x_data
                y[i] = y_data 
                
                # add 90 degrees rotation 
                X[i+self.batch_size,] = np.rot90(x_data)
                y[i+self.batch_size] = np.rot90(y_data)  
                
                # add 180 degrees rotation 
                X[i+self.batch_size*2,] = np.rot90(np.rot90(x_data)) 
                X[i+self.batch_size*2] = np.rot90(np.rot90(y_data))
                        
                # add horizontal flip 
                X[i+self.batch_size*3,] = np.fliplr(x_data)
                y[i+self.batch_size*3] = np.fliplr(y_data) 
                
                # add vertical filp 
                X[i+self.batch_size*4,] = np.flipud(x_data) 
                y[i+self.batch_size*4] = np.flipud(y_data)
            
            # shuffle once more to make training harder 
            X,y = shuffle(X,y) 
            return (X, y)
        
        else: 
            # Initialization
            X = np.empty((self.batch_size, 120, 120, 4))
            y = np.empty((self.batch_size, 120, 120, 1)) 

            # Generate data
            for i, ID in enumerate(list_IDs_temp):
                data = np.load('./storage/precipitation/train/' + ID)
                # Store sample
                X[i,] = data[:,:,:4] 
                y[i] = data[:,:,-1].reshape((120,120,1)) 
            
            return (X, y) 


In [21]:
# uses skip connections and also adds information from both MaxPooling2D and AveragePooling2D 
def conv2d_block(input_layer, n_filters, kernel):
    conv1 = Conv2D(n_filters, kernel, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(input_layer)
    conv1 = BatchNormalization()(conv1)
    conv2 = Conv2D(n_filters, kernel, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1) 
    conv2 = BatchNormalization()(conv2)
    conv3 = Conv2D(n_filters, kernel, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2) 
    conv3 = Add()([conv3, conv1])   
    conv3 = BatchNormalization()(conv3) 
    maxpool = MaxPooling2D((2,2))(conv3) 
    avgpool = AveragePooling2D((2,2))(conv3)
    ret = Add()([maxpool,avgpool]) 
    return conv3, ret # conv3: for UNet concatenation, ret: to pass into succeeding layers 

def uconv2d_block(input_layer, n_filters, kernel): 
    conv1 = Conv2D(n_filters, kernel, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(input_layer)
    conv1 = BatchNormalization()(conv1) 
    conv2 = Conv2D(n_filters, kernel, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1) 
    conv2 = BatchNormalization()(conv2) 
    conv3 = Conv2D(n_filters, kernel, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2) 
    conv3 = Add()([conv3,conv1]) 
    conv3 = BatchNormalization()(conv3) 
    return conv3 

# this version takes too long to train 
def build_model(start_neurons):
    
    inputs = Input((120,120,4)) 
    
    bn = BatchNormalization()(inputs)
    
    # compressing stages - skip connections with multiple kernel sizes.  
    conv11, pooled11 = conv2d_block(bn, start_neurons, 1)
    conv21, pooled21 = conv2d_block(bn, start_neurons, 3) 
    conv31, pooled31 = conv2d_block(bn, start_neurons, 5) 
    conv1 = concatenate([conv11,conv21,conv31]) 
    pooled1 = concatenate([pooled11,pooled21,pooled21])
    
    
    conv12, pooled12 = conv2d_block(pooled1, start_neurons * 2, 1)
    conv22, pooled22 = conv2d_block(pooled1, start_neurons * 2, 3) 
    conv32, pooled32 = conv2d_block(pooled1, start_neurons * 2, 5) 
    conv2 = concatenate([conv12,conv22,conv32]) 
    pooled2 = concatenate([pooled12,pooled22,pooled32]) 
    
    conv13, pooled13 = conv2d_block(pooled2, start_neurons * 4, 1) 
    conv23, pooled23 = conv2d_block(pooled2, start_neurons * 4, 3) 
    conv33, pooled33 = conv2d_block(pooled2, start_neurons * 4, 5) 
    conv3 = concatenate([conv13,conv23,conv33]) 
    pooled3 = concatenate([pooled13,pooled23,pooled33])  

    # middle convolutional layer, keeping it simple 
    convm = Conv2D(start_neurons * 8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pooled3)
    
    # decompressing stage - skip connections with multiple kernel sizes 
    deconv3 = Conv2DTranspose(start_neurons * 4, 3, strides = 2, padding = 'same')(convm)
    uconv3 = concatenate([deconv3,conv3])
    uconv13 = uconv2d_block(uconv3, start_neurons * 4, 1)
    uconv23 = uconv2d_block(uconv3, start_neurons * 4, 3) 
    uconv33 = uconv2d_block(uconv3, start_neurons * 4, 5)
    uconv3 = concatenate([uconv13,uconv23,uconv33]) 
    
    deconv2 = Conv2DTranspose(start_neurons * 2, 3, strides = 2, padding = 'same')(uconv3) 
    uconv2 = concatenate([deconv2,conv2]) 
    uconv12 = uconv2d_block(uconv2, start_neurons * 2, 1) 
    uconv22 = uconv2d_block(uconv2, start_neurons * 2, 3)
    uconv32 = uconv2d_block(uconv2, start_neurons * 2, 5) 
    uconv2 = concatenate([uconv12,uconv22,uconv32]) 
    
    deconv1 = Conv2DTranspose(start_neurons, 3, strides = 2, padding = 'same')(uconv2) 
    uconv1 = concatenate([deconv1,conv1]) 
    uconv11 = uconv2d_block(uconv1, start_neurons, 1)
    uconv21 = uconv2d_block(uconv1, start_neurons, 3) 
    uconv31 = uconv2d_block(uconv1, start_neurons, 5) 

    outputs = Conv2D(1, (1,1), padding="same", activation='relu')(uconv1)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='mae',optimizer='adam')
    return model


def build_model_2(start_neurons):
    
    inputs = Input((120,120,4)) 
    
    bn = BatchNormalization()(inputs)
    
    # compressing stages - skip connections with multiple kernel sizes.  
    conv1, pooled1 = conv2d_block(bn, start_neurons, 3) 
    conv2, pooled2 = conv2d_block(pooled1, start_neurons * 2, 3)
    conv3, pooled3 = conv2d_block(pooled2, start_neurons * 4, 3) 

    # middle convolutional layer, keeping it simple 
    convm = Conv2D(start_neurons * 8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pooled3)
    
    # decompressing stage - skip connections with multiple kernel sizes 
    deconv3 = Conv2DTranspose(start_neurons * 4, 3, strides = 2, padding = 'same')(convm)
    uconv3 = concatenate([deconv3,conv3])
    uconv3 = uconv2d_block(uconv3, start_neurons * 4, 3) 
    
    deconv2 = Conv2DTranspose(start_neurons * 2, 3, strides = 2, padding = 'same')(uconv3) 
    uconv2 = concatenate([deconv2,conv2])
    uconv2 = uconv2d_block(uconv2, start_neurons * 2, 3)
    
    deconv1 = Conv2DTranspose(start_neurons, 3, strides = 2, padding = 'same')(uconv2) 
    uconv1 = concatenate([deconv1,conv1]) 
    uconv1 = uconv2d_block(uconv1, start_neurons, 3) 

    outputs = Conv2D(1, (1,1), padding="same", activation='relu')(uconv1)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='mae',optimizer='adam')
    return model


def build_model_3(start_neurons):
    
    inputs = Input((120,120,4)) 
    
    bn = BatchNormalization()(inputs)
    
    # compressing stages - skip connections with multiple kernel sizes.  
    conv1, pooled1 = conv2d_block(bn, start_neurons, 3) 
    conv2, pooled2 = conv2d_block(pooled1, start_neurons * 2, 3)

    # middle convolutional layer, keeping it simple 
    convm = Conv2D(start_neurons * 4, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pooled2)
    
    # decompressing stage - skip connections with multiple kernel sizes     
    deconv2 = Conv2DTranspose(start_neurons * 2, 3, strides = 2, padding = 'same')(convm) 
    uconv2 = concatenate([deconv2,conv2])
    uconv2 = uconv2d_block(uconv2, start_neurons * 2, 3)
    
    deconv1 = Conv2DTranspose(start_neurons, 3, strides = 2, padding = 'same')(uconv2) 
    uconv1 = concatenate([deconv1,conv1]) 
    uconv1 = uconv2d_block(uconv1, start_neurons, 3) 

    outputs = Conv2D(1, (1,1), padding="same", activation='relu')(uconv1)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='mae',optimizer='adam')
    return model

def build_resnet(middle_depth):
    
    inputs = Input((120,120,4)) 
    bn = BatchNormalization()(inputs)
    conv0 = Conv2D(32, kernel_size=1, strides=1, padding='same', activation='relu')(bn)

    bn = BatchNormalization()(conv0)
    conv = Conv2D(32, kernel_size=2, strides=1, padding='same', activation='relu')(bn)
    concat = concatenate([conv0, conv], axis=3)

    bn = BatchNormalization()(concat)
    conv = Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu')(bn)
    concat = concatenate([concat, conv], axis=3)

    for i in range(middle_depth):
        bn = BatchNormalization()(concat)
        conv = Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu')(bn)
        concat = concatenate([concat, conv], axis=3)

    bn = BatchNormalization()(concat)
    outputs = Conv2D(1, kernel_size=1, strides=1, padding='same', activation='relu')(bn)

    model = Model(inputs=inputs, outputs=outputs)

    return model

def base_UNET(start_neurons):
    
    inputs = Input((120,120,4))
    bn = BatchNormalization()(inputs)
    
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(bn)
    pool1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D((2, 2))(pool1)

    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(pool1)
    pool2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D((2, 2))(pool2)

    convm = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(pool2)

    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)
    uconv2 = BatchNormalization()(uconv2)

    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    uconv1 = BatchNormalization()(uconv1)
    outputs = Conv2D(1, (1,1), padding="same", activation='relu')(uconv1) 
    
    model = Model(inputs =inputs, outputs=outputs) 
    model.compile(loss='mae',optimizer='adam')
    
    return model

def base_UNET2(start_neurons): 
    inputs = Input((120,120,4))
    bn = BatchNormalization()(inputs)
    
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(bn)
    pool1 = BatchNormalization()(conv1)
    maxpool1 = MaxPooling2D((2, 2))(pool1) 
    avgpool1 = AveragePooling2D((2,2))(pool1) 
    pool1 = Add()([maxpool1,avgpool1]) 

    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(pool1)
    pool2 = BatchNormalization()(conv2)
    maxpool2 = MaxPooling2D((2, 2))(pool2) 
    avgpool2 = AveragePooling2D((2,2))(pool2) 
    pool2 = Add()([maxpool2,avgpool2]) 

    convm = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(pool2)

    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)
    uconv2 = BatchNormalization()(uconv2)

    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    uconv1 = BatchNormalization()(uconv1)
    outputs = Conv2D(1, (1,1), padding="same", activation='relu')(uconv1) 
    
    model = Model(inputs =inputs, outputs=outputs) 
    model.compile(loss='mae',optimizer='adam')
    
    return model
    


In [ ]:
def base_UNET3(start_neurons): 
    inputs = Input((120,120,4))
    bn = BatchNormalization()(inputs)
    
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(bn) 
    conv12 = Conv2D(start_neurons * 1, 5, activation = 'relu', padding = 'same')(bn) 
    conv1 = concatenate([conv1,conv12]) 
    pool1 = BatchNormalization()(conv1)
    maxpool1 = MaxPooling2D((2, 2))(pool1) 
    avgpool1 = AveragePooling2D((2,2))(pool1) 
    pool1 = Add()([maxpool1,avgpool1]) 

    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(pool1)
    pool2 = BatchNormalization()(conv2)
    maxpool2 = MaxPooling2D((2, 2))(pool2) 
    avgpool2 = AveragePooling2D((2,2))(pool2) 
    pool2 = Add()([maxpool2,avgpool2]) 

    convm = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(pool2)

    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)
    uconv2 = BatchNormalization()(uconv2)

    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    uconv1 = BatchNormalization()(uconv1)
    outputs = Conv2D(1, (1,1), padding="same", activation='relu')(uconv1) 
    
    model = Model(inputs =inputs, outputs=outputs) 
    model.compile(loss='mae',optimizer='adam')
    
    return model
    


In [ ]:
train_files = [x for x in os.listdir('./storage/precipitation/train/')] 

def k_fold(k,files):  
    folds = [] 
    fold_size = len(files) // k 
    for i in range(k): 
        if i == k-1:  
            l = files[i*fold_size:] 
        else: 
            l = files[i*fold_size:(i+1)*fold_size]  
        folds.append(l)   
    return folds  

train_files = shuffle(train_files) # shuffle train files before splitting them into a fold 
train_folds = k_fold(5, train_files)

for i in range(5): 
    print("........ Fold {} Training ........".format(i+1)) 
        
    
    # split data in train and validations et 
    td = train_folds[:i] + train_folds[i+1:] 
    train_data = [] 
    for j in td: 
        for name in j: 
            train_data.append(name)
    val_data = train_folds[i] 
    
    # create partition dictionary and parameter dictionary 
    partition = {'train':[], 'validation':[]} 
    params_train_gen = {'dim': (120,120),
                    'batch_size': 32,
                    'n_channels': 4,
                    'n_timesteps': 4,
                    'shuffle': True,
                    'augment_data': True} 

    params_val_gen = {'dim': (120,120), 
                  'batch_size': 32, 
                  'n_channels': 4, 
                  'n_timesteps': 4,
                  'shuffle': True,
                  'augment_data': False}


    for filename in train_data: 
        partition['train'].append(filename) 
    for filename in val_data: 
        partition['validation'].append(filename)  
        
    
    # Generators
    training_generator = DataGenerator(partition['train'], **params_train_gen)
    validation_generator = DataGenerator(partition['validation'], **params_val_gen) 
    
    # prepare model 
    model = base_UNET(64) 

    # conduct training 
    model_path = './storage/precip_unet_base/kfold' + str(i+1) + '/epoch_{epoch:03d}_val_loss_{val_loss:.3f}.h5'
    learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_loss', patience = 3, verbose = 1, factor = 0.5)
    checkpoint = ModelCheckpoint(filepath = model_path, monitor = 'val_loss', verbose = 1, save_best_only = True)
    early_stopping = EarlyStopping(monitor = 'val_loss', patience = 10) 
    history = model.fit_generator(generator = training_generator, validation_data = validation_generator, epochs = 100, callbacks = [checkpoint, early_stopping, learning_rate_reduction])
    
    

........ Fold 1 Training ........
Epoch 1/100
1567/1568 [============================>.] - ETA: 0s - loss: 5.9419
Epoch 00001: val_loss improved from inf to 3.13009, saving model to ./storage/precip_unet_base/kfold1/epoch_001_val_loss_3.130.h5
1568/1568 [==============================] - 1247s 796ms/step - loss: 5.9411 - val_loss: 3.1301
Epoch 2/100
 837/1568 [===============>..............] - ETA: 9:24 - loss: 5.0915